In [78]:
# @title Upload CSV
import pandas as pd
import random

hari_df = pd.read_csv('data_skripsi_hari.csv')
ruang_df = pd.read_csv('data_skripsi_ruang.csv')
jam_df = pd.read_csv('data_skripsi_jam.csv')
dosen_df = pd.read_csv('data_skripsi_dosen.csv')
mk_genap_df = pd.read_csv('data_skripsi_mk_genap.csv')
data_dosen_df = pd.read_csv('data_skripsi_data_dosen.csv')

In [79]:
# @title Gabungkan data_dosen dengan dosen
merged_df = pd.merge(
    pd.merge(data_dosen_df, dosen_df, on='id_dosen'),
    mk_genap_df, on='id_mk_genap'
)

print("Data Gabungan:")
print(merged_df.head())

total_sks = merged_df['sks'].sum()
print(f"Total slot SKS yang dibutuhkan: {total_sks}")
print(f"Total data dalam merged_df: {len(merged_df)}")

Data Gabungan:
   id_dosen  id_mk_genap kelas                   nama_dosen  \
0         2    211840231     A  Ahmad Azhari, S.Kom., M.Eng   
1         2    211861331     A  Ahmad Azhari, S.Kom., M.Eng   
2         2    211861431     A  Ahmad Azhari, S.Kom., M.Eng   
3         3    211840131     A     Ali Tarmuji, S.T., M.Cs.   
4         3    211860330     A     Ali Tarmuji, S.T., M.Cs.   

                              nama_mk_genap  smt  sks    sifat kategori  \
0                          Grafika Komputer    4    3    Wajib        -   
1                Pengembangan Aplikasi Game    6    3  Pilihan       SC   
2                          Pengenalan Pola     6    3  Pilihan       SC   
3  Analisis dan Perancangan Perangkat Lunak    4    3    Wajib        -   
4                  Rekayasa Perangkat Lunak    6    3    Wajib        -   

    metode  
0  Offline  
1  Offline  
2   Online  
3  Offline  
4  Offline  
Total slot SKS yang dibutuhkan: 282
Total data dalam merged_df: 107


Tahapan GWO

1. Preprocessing
    
    Membangun struktur slot waktu.

2. Inisialisasi Populasi (GWO)
    

    Inisialisasi populasi serigala acak (Xi).

    Setiap "serigala" dalam GWO mewakili solusi penjadwalan yang mungkin (misalnya: variasi pengaturan slot).

3. Fitness Function (GWO)
    Hitung nilai fitness untuk setiap serigala.

    Mengevaluasi kualitas penjadwalan (misalnya: minimalisasi konflik, kepadatan ruang, dll).

4. Proses Optimasi (GWO)
    Tentukan Alpha, Beta, dan Delta berdasarkan nilai fitness.

    Menggunakan hierarki Alpha, Beta, Delta untuk memperbarui posisi solusi.

    While (iterasi < maksimum iterasi):
      1. Perbarui parameter a, A, dan C.
      2. For setiap serigala:
          1. Perbarui posisi serigala berdasarkan Alpha, Beta, dan Delta.
      3. Hitung nilai fitness untuk setiap serigala.
      4. Perbarui Alpha, Beta, dan Delta.
5. Postprocessing
    Return solusi terbaik (Alpha).
    
    Menampilkan jadwal terbaik hasil optimasi.


In [80]:
# @title Preprosessing
hari_list = hari_df['nama_hari'].tolist()
ruang_list = ruang_df['nama_ruang'].tolist()
jam_list = jam_df[['id_jam', 'jam_awal', 'jam_akhir']].to_dict('records')
mata_kuliah_list = mk_genap_df.set_index('id_mk_genap').to_dict('index')

def slot_generator():
    wolf = []
    for id_counter, (hari, ruang, jam) in enumerate(
        [(h, r, j) for h in hari_list for r in ruang_list for j in jam_list], start=1
    ):
        wolf.append({
            "id_slot": id_counter,
            "mata_kuliah": None,
            "dosen": None,
            "ruang": ruang,
            "hari": hari,
            "jam_mulai": jam['jam_awal'],
            "jam_selesai": jam['jam_akhir'],
            "kelas": None,
            "sks": None,
            "metode": None
        })
    return wolf

slots = slot_generator()
for slot in slots[:5]:
    print(slot)

print("\n")
for slot in slots[-5:]:
    print(slot)

{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}


{'id_slot': 500, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.63', 'hari'

In [ ]:
# @title Inisialisasi Populasi (GWO)
from collections import defaultdict
from datetime import datetime

def time_to_minutes(t):
    try:
        return datetime.strptime(t, "%H:%M:%S").hour * 60 + datetime.strptime(t, "%H:%M:%S").minute
    except ValueError:
        return datetime.strptime(t, "%H:%M").hour * 60 + datetime.strptime(t, "%H:%M").minute

def create_random_schedule():
    schedule = slot_generator()
    merged_shuffled = merged_df.sample(frac=1).iterrows()
    
    # Struktur tracking untuk hard constraints
    room_schedule = defaultdict(list)
    teacher_schedule = defaultdict(list)
    day_schedule = defaultdict(list)

    # total_data=0

    for index, row in merged_shuffled:
        mata_kuliah = row['nama_mk_genap']
        dosen = row['nama_dosen']
        kelas = row['kelas']
        sks = int(row['sks'])
        metode = row['metode']

        possible_positions = list(range(len(schedule) - sks + 1))
        random.shuffle(possible_positions)

        placed = False
        for i in possible_positions:
            block = schedule[i:i+sks]
            
            # Validasi dasar slot
            all_empty = all(slot['mata_kuliah'] is None for slot in block)
            same_hari = len({slot['hari'] for slot in block}) == 1
            same_ruang = (metode == 'Online') or (len({slot['ruang'] for slot in block}) == 1)
            
            if not (all_empty and same_ruang and same_hari):
                continue
                
            # Ekstrak informasi waktu
            target_hari = block[0]['hari']
            target_ruang = block[0]['ruang'] if metode != 'Online' else None
            block_start = time_to_minutes(block[0]['jam_mulai'])
            block_end = time_to_minutes(block[-1]['jam_selesai'])
            
            # Hard constraint 1: Cek konflik ruang
            room_conflict = False
            if metode != 'Online':
                for existing_start, existing_end in room_schedule.get((target_ruang, target_hari), []):
                    if not (block_end <= existing_start or block_start >= existing_end):
                        room_conflict = True
                        break

            # Hard constraint 2: Cek konflik dosen
            teacher_conflict = False
            for existing_start, existing_end in teacher_schedule.get((dosen, target_hari), []):
                if not (block_end <= existing_start or block_start >= existing_end):
                    teacher_conflict = True
                    break

            # Hard constrain 3: Cek konflik hari
            day_conflict = False
            for existing_start, existing_end in day_schedule.get((dosen, target_hari), []):
                if not (block_end <= existing_start or block_start >= existing_end):
                    day_conflict = True
                    break
                
            if not room_conflict and not teacher_conflict and not day_conflict:
                
                ruang = 'Online' if metode == 'Online' else target_ruang
                for slot in block:
                    slot.update({
                        "mata_kuliah": mata_kuliah,
                        "dosen": dosen,
                        "kelas": kelas,
                        "sks": sks,
                        "metode": metode,
                        "ruang": ruang
                    })
                # Update tracking
                if metode != 'Online':
                    room_schedule[(target_ruang, target_hari)].append((block_start, block_end))
                    teacher_schedule[(dosen, target_hari)].append((block_start, block_end))
                placed = True
                # total_data += 1
                break
                
        if not placed:
            pass
        # print(total_data)
    return schedule

population_size = 1
population = [create_random_schedule() for _ in range(population_size)]

for i in range(population_size):
    print(f"\nIndividu {i+1}:")
    print(population[i])

    for slot in population[i]:
        print(slot)



Individu 1:
[{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}, {'id_slot': 2, 'mata_kuliah': 'Kriptografi', 'dosen': 'Eko Aribowo, S.T., M.Kom.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}, {'id_slot': 3, 'mata_kuliah': 'Kriptografi', 'dosen': 'Eko Aribowo, S.T., M.Kom.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}, {'id_slot': 4, 'mata_kuliah': 'Kriptografi', 'dosen': 'Eko Aribowo, S.T., M.Kom.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}, {'id_slot': 5, 'mata_kuliah': 'Grafika Komputer', 'dosen': 'Ahmad Azhari, S.Kom., M.Eng', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00

Konstrain berat 1
Konstrain ringan 0.5

Konstrain berat:
jadwal dosen mengajar kelas yang berbeda pada jam yang sama
jadwal ruang kelas ruang kelas digunakan lebih dari 1 dosen 

Konstrain ringan:
dosen mengajar pada hari yang sama
kelas dijadwalkan pada waktu paling akhir dan paling awal pada hari itu

In [82]:
# @title Fitness Function (GWO)
from collections import defaultdict

def minutes_to_time(mins):
    hours = mins // 60
    minutes = mins % 60
    return f"{hours:02d}:{minutes:02d}"

def calculate_fitness(schedule):
    max_score = 100
    penalty = 0

    teacher_conflicts = []
    room_conflicts = []
    
    # Struktur tracking untuk semua konstrain
    teacher_schedule = defaultdict(list)
    room_schedule = defaultdict(list)
    teacher_daily_courses = defaultdict(set)
    class_time_bounds = defaultdict(lambda: {'min': float('inf'), 'max': -float('inf')})
    penalized_classes = set()

    # Fungsi untuk menghitung overlap
    def count_overlaps(intervals):
        intervals.sort()
        overlaps = 0
        for i in range(1, len(intervals)):
            if intervals[i][0] < intervals[i-1][1]:
                overlaps += 1
        return overlaps

    # First pass: Kumpulkan semua data
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
            
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        hari = slot['hari']
        dosen = slot['dosen']
        ruang = slot['ruang']
        kelas = slot['kelas']
        mk = slot['mata_kuliah']

        # Track untuk hard constraints
        teacher_schedule[(dosen, hari)].append((start, end))
        if ruang != 'Online':
            room_schedule[(ruang, hari)].append((start, end))

        # Track untuk soft constraints
        teacher_daily_courses[(dosen, hari)].add(mk)
        class_time_bounds[(kelas, hari)]['min'] = min(class_time_bounds[(kelas, hari)]['min'], start)
        class_time_bounds[(kelas, hari)]['max'] = max(class_time_bounds[(kelas, hari)]['max'], start)

    # Hitung hard constraints
    # 1. Dosen mengajar di kelas berbeda pada jam yang sama
    for (dosen, hari), intervals in teacher_schedule.items():
        sorted_intervals = sorted(intervals)
        for i in range(1, len(sorted_intervals)):
            prev = sorted_intervals[i-1]
            current = sorted_intervals[i]
            if current[0] < prev[1]:
                teacher_conflicts.append({
                    'teacher': dosen,
                    'day': hari,
                    'time1_start': prev[0],
                    'time1_end': prev[1],
                    'time2_start': current[0],
                    'time2_end': current[1]
                })
                penalty += 1
        
    # 2. Ruang digunakan oleh lebih dari 1 kelas di waktu sama
    for (ruang, hari), intervals in room_schedule.items():
        sorted_intervals = sorted(intervals)
        for i in range(1, len(sorted_intervals)):
            prev = sorted_intervals[i-1]
            current = sorted_intervals[i]
            if current[0] < prev[1]:
                room_conflicts.append({
                    'room': ruang,
                    'day': hari,
                    'time1_start': prev[0],
                    'time1_end': prev[1],
                    'time2_start': current[0],
                    'time2_end': current[1]
                })
                penalty += 1

    # Hitung soft constraints
    # 1. Dosen mengajar di hari yang sama
    for courses in teacher_daily_courses.values():
        if len(courses) > 1:
            penalty += 0.5 * (len(courses) - 1)  # Ringan 0.5 per tambahan MK
            
    # 2. Kelas di jam pertama/terakhir
    for (kelas, hari), bounds in class_time_bounds.items():
        if (kelas, hari) not in penalized_classes:
            min_time = bounds['min']
            max_time = bounds['max']
            
            # Cek keberadaan slot di jam awal/akhir
            has_min = any(
                time_to_minutes(slot['jam_mulai']) == min_time 
                for slot in schedule 
                if slot['kelas'] == kelas 
                and slot['hari'] == hari 
                and slot['mata_kuliah']
            )
            
            has_max = any(
                time_to_minutes(slot['jam_mulai']) == max_time 
                for slot in schedule 
                if slot['kelas'] == kelas 
                and slot['hari'] == hari 
                and slot['mata_kuliah']
            )
            
            if has_min:
                penalty += 0.5
            if has_max:
                penalty += 0.5
                
            penalized_classes.add((kelas, hari))

    return {
        'score': max(0, max_score - penalty),
        'penalty': penalty,
        'violations': {
            'hard_teacher': len(teacher_conflicts),
            'hard_room': len(room_conflicts),
            'soft_teaching_day': sum(len(v)-1 for v in teacher_daily_courses.values() if len(v) > 1),
            'soft_edge_time': sum(1 for _ in penalized_classes)
        },
        'teacher_conflicts': teacher_conflicts,
        'room_conflicts': room_conflicts
    }

# Generate populasi dan tampilkan hasil
population_size = 1
population = [create_random_schedule() for _ in range(population_size)]

for idx, schedule in enumerate(population):
    fitness = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1}:")
    
    # Tampilkan jadwal
    for slot in schedule:  # Perbaikan dari population[i] menjadi schedule
        print(slot)
    
    # Tampilkan fitness
    print(f"\nFitness: {fitness['score']}")
    print(f"Total Penalty: {fitness['penalty']}")
    
    # Tampilkan konflik dosen
    print("\nKonflik Dosen:")
    if not fitness['teacher_conflicts']:
        print("- Tidak ada konflik")
    else:
        for conflict in fitness['teacher_conflicts']:
            t1_start = minutes_to_time(conflict['time1_start'])
            t1_end = minutes_to_time(conflict['time1_end'])
            t2_start = minutes_to_time(conflict['time2_start'])
            t2_end = minutes_to_time(conflict['time2_end'])
            print(f"- Dosen {conflict['teacher']} di hari {conflict['day']}:")
            print(f"  • {t1_start}-{t1_end} bertabrakan dengan")
            print(f"  • {t2_start}-{t2_end}")

    # Tampilkan konflik ruang
    print("\nKonflik Ruang:")
    if not fitness['room_conflicts']:
        print("- Tidak ada konflik")
    else:
        for conflict in fitness['room_conflicts']:
            t1_start = minutes_to_time(conflict['time1_start'])
            t1_end = minutes_to_time(conflict['time1_end'])
            t2_start = minutes_to_time(conflict['time2_start'])
            t2_end = minutes_to_time(conflict['time2_end'])
            print(f"- Ruang {conflict['room']} di hari {conflict['day']}:")
            print(f"  • {t1_start}-{t1_end} bertabrakan dengan")
            print(f"  • {t2_start}-{t2_end}")

    # Tampilkan pelanggaran lainnya
    print("\nDetail Pelanggaran Lain:")
    print(f"- Dosen multi mata kuliah/hari: {fitness['violations']['soft_teaching_day']}")
    print(f"- Kelas di jam ujung: {fitness['violations']['soft_edge_time']}")


Individu 1:
{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 6, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55

In [83]:
# @title Final Implementation with Merged Time Display
import copy
import random
from collections import defaultdict
from datetime import datetime

def time_to_minutes(t):
    try:
        return datetime.strptime(t, "%H:%M:%S").hour * 60 + datetime.strptime(t, "%H:%M:%S").minute
    except ValueError:
        return datetime.strptime(t, "%H:%M").hour * 60 + datetime.strptime(t, "%H:%M").minute

def minutes_to_time(minutes):
    hours = minutes // 60
    mins = minutes % 60
    return f"{hours:02d}:{mins:02d}:00"

def adjust_slot_times(schedule):
    adjusted = copy.deepcopy(schedule)
    course_blocks = defaultdict(list)
    
    # Kelompokkan berdasarkan atribut unik
    for idx, slot in enumerate(adjusted):
        if slot['mata_kuliah']:
            key = (slot['mata_kuliah'], slot['kelas'], slot['dosen'], slot['hari'])
            course_blocks[key].append((idx, slot))
    
    for key, slot_data in course_blocks.items():
        slots = [s for idx, s in slot_data]
        
        # Ambil waktu mulai pertama dan akhir terakhir
        start_times = [time_to_minutes(s['jam_mulai']) for s in slots]
        end_times = [time_to_minutes(s['jam_selesai']) for s in slots]
        start_min = min(start_times)
        end_max = max(end_times)
        
        start_str = minutes_to_time(start_min)
        end_str = minutes_to_time(end_max)
        
        for idx, slot in slot_data:
            adjusted[idx]['jam_mulai'] = start_str
            adjusted[idx]['jam_selesai'] = end_str
            
    return adjusted

def repair_schedule(schedule):
    repaired = adjust_slot_times(schedule)
    teacher_map = defaultdict(list)
    room_map = defaultdict(list)
    
    # Proses per blok mata kuliah
    course_blocks = defaultdict(list)
    for slot in repaired:
        if slot['mata_kuliah']:
            key = (slot['mata_kuliah'], slot['kelas'], slot['dosen'], slot['hari'])
            course_blocks[key].append(slot)
    
    new_schedule = [s.copy() for s in repaired if not s['mata_kuliah']]
    
    for key, slots in course_blocks.items():
        conflict = False
        start = time_to_minutes(slots[0]['jam_mulai'])
        end = time_to_minutes(slots[0]['jam_selesai'])
        
        # Cek konflik untuk seluruh blok
        for interval in teacher_map[(slots[0]['dosen'], slots[0]['hari'])]:
            if interval['start'] < end and interval['end'] > start:
                conflict = True
                break
                
        if not conflict and slots[0]['ruang'] != 'Online':
            for interval in room_map[(slots[0]['ruang'], slots[0]['hari'])]:
                if interval['start'] < end and interval['end'] > start:
                    conflict = True
                    break
                    
        if not conflict:
            new_schedule.extend(slots)
            teacher_map[(slots[0]['dosen'], slots[0]['hari'])].append({'start': start, 'end': end})
            if slots[0]['ruang'] != 'Online':
                room_map[(slots[0]['ruang'], slots[0]['hari'])].append({'start': start, 'end': end})
        else:
            # Cari slot kosong untuk seluruh blok
            found = False
            for i in range(len(new_schedule)-len(slots)+1):
                if all(not s['mata_kuliah'] for s in new_schedule[i:i+len(slots)]):
                    for j in range(len(slots)):
                        new_schedule[i+j] = copy.deepcopy(slots[j])
                    found = True
                    break
            if not found:
                new_schedule.extend([{
                    **s,
                    'mata_kuliah': None,
                    'dosen': None,
                    'kelas': None
                } for s in slots])
    
    return adjust_slot_times(new_schedule)

def update_wolf_position(wolf, alpha, beta, delta, a):
    new_wolf = copy.deepcopy(wolf)
    wolf_slots = {s['id_slot']: s for s in new_wolf}
    
    # Tingkatkan eksplorasi
    for leader in [alpha, beta, delta]:
        for slot in leader:
            if slot['mata_kuliah'] and random.random() < 0.7:
                if slot['id_slot'] in wolf_slots:
                    wolf_slots[slot['id_slot']] = copy.deepcopy(slot)
    
    return list(wolf_slots.values())

def print_schedule(schedule):
    printed = set()
    for slot in schedule:
        if slot['mata_kuliah']:
            key = (slot['mata_kuliah'], slot['kelas'], slot['dosen'], slot['hari'])
            if key not in printed:
                start = datetime.strptime(slot['jam_mulai'], "%H:%M:%S").strftime("%H:%M")
                end = datetime.strptime(slot['jam_selesai'], "%H:%M:%S").strftime("%H:%M")
                print(f"{slot['hari']} {start}-{end} ({slot['sks']} SKS) | {slot['ruang']} | {slot['mata_kuliah']} ({slot['dosen']})")
                printed.add(key)

# Eksekusi
best_schedule, _ = gwo_optimize(population, max_iter=20)
print("\n=== Jadwal Final ===")
print_schedule(best_schedule)

NameError: name 'gwo_optimize' is not defined